In [1]:
import numpy as np
import json
import os, sys, pathlib
import matplotlib
import matplotlib.pyplot as plt

In [2]:
bench_path = pathlib.Path("./output/benchmark/")
bench_path

PosixPath('output/benchmark')

In [3]:
experiments = list(bench_path.glob("*.json"))

In [4]:
c10_exp = []
for exp in experiments:
#     print(exp.name)
    if '_nPE_cifar10_' in exp.name and not 'sMlp' in exp.name:
        c10_exp.append(exp)
c10_exp

[PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch2_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch2_l8_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch4_l12_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch2_l12_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch1_l4_att_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch8_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch2_l4_att_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch1_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch1_l12_att_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch4_l12_att_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar10_patch4_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_c

In [5]:
import pandas as pd

In [6]:
exp_dict = {'ps':[], 'la':[], 'sAtt':[], 'acc':[], 'mem':[], 'time':[], 'params':[]}
for file in c10_exp:
    ## filter out name from filename
    fn = file.name
    splits = fn.split('_')
    ps = int(splits[4][5:])
    la = int(splits[5][1:])
    sAtt = (splits[6] == 'sAtt')
    
    with open(file, 'r') as f:
        data = json.load(f)
        
    best_acc = np.array(data['test_stat'])[:, 2].max()
    params = data['params']
    
#     print(ps, la, sAtt,  best_acc)
    exp_dict['ps'].append(ps)
    exp_dict['la'].append(la)    
    exp_dict['sAtt'].append(sAtt)
    exp_dict['acc'].append(best_acc)
    exp_dict['params'].append(params)
    
    ### search for benchmark memory and time in a table
    with open("./output/benchmark_memory_data.json", 'r') as f:
        data2 = json.load(f)
        
    mem, time = -1, -1
    try:
        data2 = data2[fn[:-10]]
        mem = data2['memory']
        time = data2['time_train']*1000
    except:
        pass
    exp_dict['mem'].append(mem)
    exp_dict['time'].append(time)
    
exp_df = pd.DataFrame(exp_dict)
exp_df.sort_values('ps', axis=0)

,ps,la,sAtt,acc,mem,time,params
8,1,12,False,65.30,-1,-1.000000,1057290
16,1,12,True,75.36,-1,-1.000000,1057290
4,1,4,False,64.80,8629,125.536494,789514
7,1,4,True,72.82,1181,25.504122,789514
15,2,12,False,78.01,-1,-1.000000,566346
13,2,8,False,78.26,1297,24.850798,432458
0,2,4,True,78.95,519,11.551790,298570
3,2,12,True,81.81,-1,-1.000000,566346
1,2,8,True,80.57,661,21.335354,432458
6,2,4,False,77.21,911,12.661023,298570


In [7]:
with open(filename, 'r') as f:
    file_data = json.load(f)

NameError: name 'filename' is not defined